In [1]:
import sys
sys.path.append('/home/macul/insightface/src/symbols')
sys.path.append('/home/macul/libraries/mk_utils/mklib/nn/')

In [2]:
import fresnet
import cv2
from collections import namedtuple

import mxnet as mx
from mxconv.mxresnet import MxResNet
from mxiter.mxiter import ImageMultiLabelIter,ImageMultiLabelIter_IndividualLabel
from mxloss.mxloss import MxLosses
import numpy as np
import argparse
import logging
import json
import os

def get_image(filename):
    img = cv2.imread(filename)  # read image in b,g,r order
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   # change to r,g,b order
    img = cv2.resize(img, (112, 112))  # resize to 224*224 to fit model
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)  # change to (channel, height, width)
    img = img[np.newaxis, :]  # extend to (example, channel, heigth, width)
    return img

Batch = namedtuple('Batch', ['data'])

In [3]:
#model = MxResNet.build(1000, (3, 4, 14, 3), (64, 256, 512, 1024, 2048))
model = MxResNet.build(512, (3, 4, 6, 3), (64, 64, 128, 256, 512))
softmax_label = mx.symbol.Variable('softmax_label')
model = MxLosses.arc_loss(	model, 
                                    softmax_label,
                            95000,
                          512,
                                    0.5, 
                                    64.0, 
                          1.0)
                                  

In [4]:
trainIter = mx.io.ImageRecordIter(
									path_imgrec='/media/macul/black/face_database_raw_data/ms1m_crop_112x112/rec/train.rec',
									data_shape=(3,112,112),
									batch_size=1,
                                    scale = 0.0078125,
                                    label_width=11)

In [5]:
trainIter = ImageMultiLabelIter_IndividualLabel(trainIter, label_idx=0, label_len=[1, 10], label_name=['softmax_label', 'landmark_gt'])

In [5]:
batch=trainIter.next()

In [6]:
batch.label[0]


[[ 8.6772000e+04 -2.0261787e-02  1.9722117e-02  1.9686745e-02
  -6.5125977e-03  6.2154531e-02  7.1174796e-03 -3.1467929e-02
  -2.2152914e-02  2.6997736e-02 -4.5998152e-02]]
<NDArray 1x11 @cpu_pinned(0)>

In [6]:
i=0
while True:
    trainIter.next()
    i+=1
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


StopIteration: 

In [35]:
batch.data[0]


[[[[1.6171875 1.6328125 1.6171875 ... 1.609375  1.6328125 1.6328125]
   [1.6328125 1.640625  1.6328125 ... 1.59375   1.6171875 1.640625 ]
   [1.609375  1.6171875 1.625     ... 1.5859375 1.609375  1.6328125]
   ...
   [1.625     1.625     1.625     ... 1.546875  1.6328125 1.625    ]
   [1.6171875 1.625     1.625     ... 1.5859375 1.6484375 1.65625  ]
   [1.609375  1.609375  1.6171875 ... 1.46875   1.5703125 1.6484375]]

  [[1.625     1.640625  1.6171875 ... 1.625     1.6328125 1.6171875]
   [1.640625  1.6484375 1.6328125 ... 1.609375  1.6171875 1.625    ]
   [1.609375  1.6171875 1.625     ... 1.59375   1.609375  1.6328125]
   ...
   [1.640625  1.640625  1.640625  ... 1.5859375 1.625     1.59375  ]
   [1.6328125 1.640625  1.640625  ... 1.625     1.640625  1.625    ]
   [1.625     1.625     1.6328125 ... 1.5234375 1.5625    1.6171875]]

  [[1.640625  1.65625   1.6328125 ... 1.6171875 1.6328125 1.625    ]
   [1.65625   1.6640625 1.6484375 ... 1.6015625 1.6171875 1.6328125]
   [1.609375  1

In [37]:
print(trainIter.iter_next())

None


In [6]:
all_layers = model.get_internals()
all_layers.list_outputs()

['data',
 'stem_conv1_weight',
 'stem_conv1_output',
 'stem_bn2_gamma',
 'stem_bn2_beta',
 'stem_bn2_moving_mean',
 'stem_bn2_moving_var',
 'stem_bn2_output',
 'stem_relu1_gamma',
 'stem_relu1_output',
 'stage1_unit1_bn1_gamma',
 'stage1_unit1_bn1_beta',
 'stage1_unit1_bn1_moving_mean',
 'stage1_unit1_bn1_moving_var',
 'stage1_unit1_bn1_output',
 'stage1_unit1_prelu1_gamma',
 'stage1_unit1_prelu1_output',
 'stage1_unit1_conv1_weight',
 'stage1_unit1_conv1_output',
 'stage1_unit1_bn2_gamma',
 'stage1_unit1_bn2_beta',
 'stage1_unit1_bn2_moving_mean',
 'stage1_unit1_bn2_moving_var',
 'stage1_unit1_bn2_output',
 'stage1_unit1_prelu2_gamma',
 'stage1_unit1_prelu2_output',
 'stage1_unit1_conv2_weight',
 'stage1_unit1_conv2_output',
 'stage1_unit1_convr_weight',
 'stage1_unit1_convr_output',
 '_plus0_output',
 'stage1_unit2_bn1_gamma',
 'stage1_unit2_bn1_beta',
 'stage1_unit2_bn1_moving_mean',
 'stage1_unit2_bn1_moving_var',
 'stage1_unit2_bn1_output',
 'stage1_unit2_prelu1_gamma',
 'stage1_u

In [7]:
sym3 = all_layers['out_relu1_output']
mod3 = mx.mod.Module(symbol=sym3, context=mx.cpu())
mod3.bind(for_training=False, data_shapes=[('data', (1,3,112,112))])
mod3.init_params()

/home/macul/mx_venv/local/lib/python2.7/site-packages/mxnet/module/base_module.py:55: UserWarning: You created Module with Module(..., label_names=['softmax_label']) but input with name 'softmax_label' is not found in symbol.list_arguments(). Did you mean one of:
	data
  warnings.warn(msg)
/home/macul/mx_venv/local/lib/python2.7/site-packages/mxnet/module/base_module.py:67: UserWarning: Data provided by label_shapes don't match names specified by label_names ([] vs. ['softmax_label'])
  warnings.warn(msg)


In [7]:
sym3 = all_layers['softmax_output']
mod3 = mx.mod.Module(symbol=sym3, label_names=['softmax_label'], context=mx.cpu())
mod3.bind(for_training=True, data_shapes=[('data', (1,3,112,112))],label_shapes=[('softmax_label', (1,1))])
mod3.init_params()

RuntimeError: simple_bind error. Arguments:
data: (1, 3, 112, 112)
softmax_label: (1, 1)
Error in operator _plus16: [14:51:08] /home/travis/build/dmlc/mxnet-distro/mxnet-build/3rdparty/mshadow/../../src/operator/tensor/../elemwise_op_common.h:133: Check failed: assign(&dattr, (*vec)[i]) Incompatible attr in node _plus16 at 1-th input: expected [1,95000], got [1,1,95000]

Stack trace returned 10 entries:
[bt] (0) /home/macul/mx_venv/local/lib/python2.7/site-packages/mxnet/libmxnet.so(+0x37f51a) [0x7fd17681a51a]
[bt] (1) /home/macul/mx_venv/local/lib/python2.7/site-packages/mxnet/libmxnet.so(+0x37fb51) [0x7fd17681ab51]
[bt] (2) /home/macul/mx_venv/local/lib/python2.7/site-packages/mxnet/libmxnet.so(+0x54078f) [0x7fd1769db78f]
[bt] (3) /home/macul/mx_venv/local/lib/python2.7/site-packages/mxnet/libmxnet.so(+0x793444) [0x7fd176c2e444]
[bt] (4) /home/macul/mx_venv/local/lib/python2.7/site-packages/mxnet/libmxnet.so(+0x2b41d8a) [0x7fd178fdcd8a]
[bt] (5) /home/macul/mx_venv/local/lib/python2.

In [9]:
img = get_image('/home/macul/2018-10-15.png')

#output = mod3(Batch([mx.nd.array(img)]))
batch = trainIter.next()
mod3.forward(batch)
#print mod.get_outputs()[0].asnumpy()
#mod3.predict(trainIter)

In [10]:
mod3.get_outputs()[0].asnumpy().shape

(1, 95000)

In [14]:
out = mod3.get_outputs()[0].asnumpy()
print(out.shape)

(1, 95000)


In [15]:
batch.label[0]


[[86772.]]
<NDArray 1x1 @cpu(0)>

In [40]:
sym = fresnet.get_symbol(1000, 50, 
        version_se=0, version_input=1, 
        version_output='E', version_unit=3,
        version_act='prelu')

0 1 E 3 prelu


In [41]:
all_layers = sym.get_internals()

In [42]:
all_layers.list_outputs()

['data',
 'id_output',
 '_minusscalar2_output',
 '_mulscalar2_output',
 'conv0_weight',
 'conv0_output',
 'bn0_gamma',
 'bn0_beta',
 'bn0_moving_mean',
 'bn0_moving_var',
 'bn0_output',
 'relu0_gamma',
 'relu0_output',
 'stage1_unit1_bn1_gamma',
 'stage1_unit1_bn1_beta',
 'stage1_unit1_bn1_moving_mean',
 'stage1_unit1_bn1_moving_var',
 'stage1_unit1_bn1_output',
 'stage1_unit1_conv1_weight',
 'stage1_unit1_conv1_output',
 'stage1_unit1_bn2_gamma',
 'stage1_unit1_bn2_beta',
 'stage1_unit1_bn2_moving_mean',
 'stage1_unit1_bn2_moving_var',
 'stage1_unit1_bn2_output',
 'stage1_unit1_relu1_gamma',
 'stage1_unit1_relu1_output',
 'stage1_unit1_conv2_weight',
 'stage1_unit1_conv2_output',
 'stage1_unit1_bn3_gamma',
 'stage1_unit1_bn3_beta',
 'stage1_unit1_bn3_moving_mean',
 'stage1_unit1_bn3_moving_var',
 'stage1_unit1_bn3_output',
 'stage1_unit1_conv1sc_weight',
 'stage1_unit1_conv1sc_output',
 'stage1_unit1_sc_gamma',
 'stage1_unit1_sc_beta',
 'stage1_unit1_sc_moving_mean',
 'stage1_unit1_sc

In [55]:
sym3 = all_layers['stage4_unit3_bn3_output']
mod3 = mx.mod.Module(symbol=sym3, label_names=None, context=mx.cpu())
mod3.bind(for_training=False, data_shapes=[('data', (1,3,448,448))])
mod3.init_params()
#mod3.set_params(arg_params, aux_params)

In [56]:

img = get_image('/home/macul/2018-10-15.png')

#output = mod3(Batch([mx.nd.array(img)]))
mod3.forward(Batch([mx.nd.array(img)]))

In [57]:

out = mod3.get_outputs()[0].asnumpy()
print(out.shape)

(1, 512, 7, 7)


In [17]:
512*7*7

25088

In [ ]:
all_label = mx.symbol.Variable('softmax_label')
gt_label = all_label
extra_loss = None
_weight = mx.symbol.Variable("fc7_weight", shape=(args.num_classes, args.emb_size), lr_mult=args.fc7_lr_mult, wd_mult=args.fc7_wd_mult)

s = args.margin_s #64
m = args.margin_m #0.5
assert s>0.0
assert m>=0.0
assert m<(math.pi/2)
_weight = mx.symbol.L2Normalization(_weight, mode='instance')
nembedding = mx.symbol.L2Normalization(embedding, mode='instance', name='fc1n')*s
fc7 = mx.sym.FullyConnected(data=nembedding, weight = _weight, no_bias = True, num_hidden=args.num_classes, name='fc7')
zy = mx.sym.pick(fc7, gt_label, axis=1)
cos_t = zy/s
cos_m = math.cos(m)
sin_m = math.sin(m)
mm = math.sin(math.pi-m)*m
#threshold = 0.0
threshold = math.cos(math.pi-m)
if args.easy_margin: #0
  cond = mx.symbol.Activation(data=cos_t, act_type='relu')
else:
  cond_v = cos_t - threshold
  cond = mx.symbol.Activation(data=cond_v, act_type='relu')
body = cos_t*cos_t
body = 1.0-body
sin_t = mx.sym.sqrt(body)
new_zy = cos_t*cos_m
b = sin_t*sin_m
new_zy = new_zy - b
new_zy = new_zy*s
if args.easy_margin:
  zy_keep = zy
else:
  zy_keep = zy - s*mm
new_zy = mx.sym.where(cond, new_zy, zy_keep)

diff = new_zy - zy
diff = mx.sym.expand_dims(diff, 1)
gt_one_hot = mx.sym.one_hot(gt_label, depth = args.num_classes, on_value = 1.0, off_value = 0.0)
body = mx.sym.broadcast_mul(gt_one_hot, diff)
fc7 = fc7+body    


out_list = [mx.symbol.BlockGrad(embedding)]
softmax = mx.symbol.SoftmaxOutput(data=fc7, label = gt_label, name='softmax', normalization='valid')
out_list.append(softmax)
if args.loss_type==6:
out_list.append(intra_loss)
if args.loss_type==7:
out_list.append(inter_loss)
#out_list.append(mx.sym.BlockGrad(counter_weight))
#out_list.append(intra_loss)
if args.ce_loss:
#ce_loss = mx.symbol.softmax_cross_entropy(data=fc7, label = gt_label, name='ce_loss')/args.per_batch_size
body = mx.symbol.SoftmaxActivation(data=fc7)
body = mx.symbol.log(body)
_label = mx.sym.one_hot(gt_label, depth = args.num_classes, on_value = -1.0, off_value = 0.0)
body = body*_label
ce_loss = mx.symbol.sum(body)/args.per_batch_size
out_list.append(mx.symbol.BlockGrad(ce_loss))
out = mx.symbol.Group(out_list)

In [4]:
model, argParams, auxParams = mx.model.load_checkpoint('/home/macul/libraries/mk_utils/mx_facerecog_resnet50/output/train_5/train_5', 1)

In [9]:
model

<Symbol group [softmax, lro, embedding]>

In [8]:
argParams.keys()

['stage4_unit2_prelu2_gamma',
 'stage3_unit2_bn3_beta',
 'stage2_unit2_bn1_beta',
 'stage3_unit5_bn3_gamma',
 'out_fc1_bias',
 'stage4_unit1_prelu3_gamma',
 'stage3_unit6_bn3_gamma',
 'stage3_unit3_conv1_weight',
 'stage2_unit1_bn3_gamma',
 'stage4_unit1_prelu1_gamma',
 'out_embedding_beta',
 'stage2_unit3_bn1_beta',
 'stage3_unit4_conv1_weight',
 'stage2_unit2_bn3_beta',
 'stage2_unit1_prelu2_gamma',
 'stage1_unit3_bn1_beta',
 'stage2_unit2_conv1_weight',
 'stage1_unit2_bn2_beta',
 'stage3_unit2_conv2_weight',
 'stage1_unit2_conv1_weight',
 'stem_bn1_gamma',
 'stage2_unit1_conv3_weight',
 'stage3_unit2_bn2_gamma',
 'stage1_unit1_conv3_weight',
 'stage1_unit1_conv1_weight',
 'out_embedding_gamma',
 'stage4_unit1_bn3_beta',
 'stage4_unit3_conv3_weight',
 'stage1_unit1_convr_weight',
 'regr_fc_weight',
 'stage2_unit2_conv2_weight',
 'stage1_unit3_bn3_beta',
 'stage2_unit4_prelu1_gamma',
 'stage3_unit6_prelu1_gamma',
 'stage1_unit3_bn2_gamma',
 'stage1_unit3_bn3_gamma',
 'stage2_unit1_pre